### Librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import date, time, datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from hdbcli import dbapi as db

from statsmodels.tsa.api import STLForecast, ExponentialSmoothing, ExponentialSmoothing

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import r2_score

from statsmodels.tsa.api import STLForecast, ExponentialSmoothing, ETSModel

import warnings
warnings.filterwarnings('ignore')

from prophet import Prophet

import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score

from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

from pmdarima.arima import auto_arima
from statsmodels.tsa.api import STLForecast
from statsmodels.tsa.arima.model import ARIMA


import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from openpyxl import Workbook

import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from openpyxl import Workbook

### Base de datos

In [2]:
conn = db.connect(address='10.0.0.110', port='30015', user='DESARROLLO', password='DevHermeco.2022')
cursor = conn.cursor()
sql ='''SELECT "AÑO",
"CANAL",
"MES",
"MATERIAL",
"AFS: COLOR",
"AFS: TALLA", 
"UNIDADES", 
"Sublínea",
"Mundo",
"Tipo de Artículo", 
"Concepto Diseño",
"Año de Venta", 
"Mes de Venta", 
"FECHA VENTA"
FROM HEP300.VW_CD_REPORTE_VENTAS_ALL_2016 \
WHERE(CANAL = 'FRANQUICIAS' OR CANAL = 'TIENDAS PROPIAS')
AND "AÑO" >= 2018
'''
df_canal = pd.read_sql_query(sql, conn)

#### Filtros

In [3]:
# canal = ['TIENDAS PROPIAS', 'FRANQUICIAS']
# mundo = 'NIÑA'
# # tipo = 'MINIFALDA'
# sublinea = 'HALLOWEEN'
tipo_modelo = 1 # tipo de modelo, si es lineal es 1. Mes a mes 0
# fecha_inicial = pd.to_datetime('2023-10-01')
# fecha_final = pd.to_datetime('2024-03-01')

# tiendas
# color
# talla

# fecha_inicial = pd.to_datetime(input('Ingrese la fecha en formato dd-mm-yyyy'))
# fecha_final = input('Ingrese la fecha en formato dd-mm-yyyy')

In [4]:
# data = df_canal.loc[df_canal['CANAL'].isin(canal)]
# data = data[data['Concepto Diseño'] == sublinea]

# data.groupby(["Mundo","Tipo de Artículo"]).count().sort_values(by="UNIDADES",ascending=False)

#### Funciones

In [5]:
# rango_fechas = pd.date_range(start=fecha_inicial, end=fecha_final, freq='MS')

# test_df = pd.DataFrame({'MES_AÑO': rango_fechas})
# test_df['MES'] = test_df['MES_AÑO'].dt.month
# test_df['AÑO'] = test_df['MES_AÑO'].dt.year
# test_df['MES_AÑO'] = test_df['MES_AÑO'].dt.strftime('%Y-%m')
# test_df['UNIDADES'] = 0
# test_df

In [6]:
def DATA_preparation(data):
        data_df = data.groupby(['MES', 'AÑO'])[
        'UNIDADES'].sum().to_frame().sort_values(by=['AÑO', 'MES']).reset_index()

        # filtrar los meses de interes OCT-MAR
        # data_df = data_df.loc[(data_df['MES'] >= fecha_inicial.month) | (data_df['MES'] <= fecha_final.month)]

        # AGREGAR COLUMNAS PRONOSTICO
        # TOMANDO LOS PRIMEROS 5 DATOS Y REEMPLAZANDO PARA EL FUTURO
        data_df['MES_AÑO'] = pd.to_datetime(data_df['AÑO'].astype(str) + '-' + data_df['MES'].astype(str)).dt.strftime('%Y-%m')
        df_model = pd.concat([data_df,test_df], axis=0, ignore_index=True)
        return df_model

In [7]:
def create_features_mes_venta(df, label=None):
    df['date'] = pd.to_datetime(df.index)
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['weekofyear'] = df['date'].dt.isocalendar().week.astype(int)
    df['covid'] = [1 if ((x > pd.Timestamp('2020-03')) & (x < pd.Timestamp('2021-03'))) else 0 for x in df.date]
    
    X = df[['quarter','month','year',
           'dayofyear','weekofyear','covid']]

    if label:
        y = df[label]
        return X, y
    return X

In [8]:
def XGB_model(df):
    fecha_inicial = '2023-03'
    fecha_final = '2024-03'

    df_model = df.copy()
    
    df_model.MES_AÑO = pd.to_datetime(df_model.MES_AÑO)
    
    #  TRAIN & TEST
    train = df_model.loc[df_model.MES_AÑO < pd.to_datetime(fecha_inicial)].set_index('MES_AÑO')
    test = df_model.loc[(df_model.MES_AÑO >= pd.to_datetime(fecha_inicial)) & 
                        (df_model.MES_AÑO < pd.to_datetime(fecha_final))].set_index('MES_AÑO')

    # Define el train y test de acuerdo con las bases de datos de train y de test
    X_train, y_train = create_features_mes_venta(train, label = 'UNIDADES')
    X_test, y_test = create_features_mes_venta(test, label = 'UNIDADES')

    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                           n_estimators = 1000,
                           early_stopping_rounds=50,
                           objective='reg:squarederror',
                           max_depth=3,
                           learning_rate=0.01,
                           eval_metric = ['rmse','mae','mape']
                           )
    
    reg.fit(X_train,y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose = False)

    pred_test = reg.predict(X_test)
    test['UNIDADES'] = np.ceil(pred_test)

    
    return test[['UNIDADES']]

In [9]:
def xgb_mes_a_mes(basicos1):
    meses_basicos = [1,2,3,9,10,11,12]
    basicos_forecast = {}

    basicos_results = pd.DataFrame()
    basicos1['MES'] = pd.to_datetime(basicos1['MES_AÑO']).dt.month

    for mes in meses_basicos:
        if mes in basicos1['MES'].unique():
            basicos_results = pd.concat([basicos_results, XGB_model(basicos1[basicos1['MES'] == mes])])
            
        else:
            basicos_forecast[mes] = 0

    return basicos_results

In [10]:
def calcular_Cv(basicos1):
    basicos_clasificacion = pd.DataFrame()
    meses = basicos1.MES.unique()
    # meses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

    # Calcular el factor CV para cada mes
    cv_mes = {}
    for mes in meses:
        if mes in basicos1['MES'].unique():
            cv_mes[mes] = round(basicos1[basicos1['MES'] == mes]['UNIDADES'].std()
                                / basicos1[basicos1['MES'] == mes]['UNIDADES'].mean(), 2)
        # else:
        #     cv_mes[mes] = 0


    lista_cv = []
    for i in cv_mes.values():
        lista_cv.append(i)
    
    coeficiente_Var = np.std(lista_cv)/np.mean(lista_cv)

    return round(coeficiente_Var,2)


#### Resultados

In [11]:
# data = df_canal.loc[df_canal['CANAL'].isin(canal)]
# data = data.loc[(data['Mundo'] == 'NIÑO') &
#                 (data['Tipo de Artículo'] == 'BUZO')]

# data

In [12]:
# basicos1 = DATA_preparation(data)

In [13]:
# basicos1.head()

In [14]:
# calcular_Cv(basicos1)

In [15]:
# XGB_model(basicos1)

In [16]:
# xgb_mes_a_mes(basicos1).sort_index()

#### Resultados BASICOS

In [17]:
basicos = df_canal[df_canal['Concepto Diseño']== 'BASICOS']

In [18]:
rango_fechas = pd.date_range(start='2023-03', end='2024-02', freq='MS')

test_df = pd.DataFrame({'MES_AÑO': rango_fechas})
test_df['MES'] = test_df['MES_AÑO'].dt.month
test_df['AÑO'] = test_df['MES_AÑO'].dt.year
test_df['MES_AÑO'] = test_df['MES_AÑO'].dt.strftime('%Y-%m')
test_df['UNIDADES'] = 0


In [19]:
# for mundo in basicos['Mundo'].unique():
    # for tipo in basicos['Tipo de Artículo'].unique():
    #     basicos_individual = basicos[(basicos['Mundo'] == mundo) &
    #                                  (basicos['Tipo de Artículo'] == tipo)]
        
    #     basicos_individual2 = basicos_individual.groupby(['MES', 'AÑO'])[
    #     'UNIDADES'].sum().to_frame().sort_values(by=['AÑO', 'MES']).reset_index()

    #     basicos_individual2['MES_AÑO'] = pd.to_datetime(basicos_individual2['AÑO'].astype(str) + 
    #                                 '-' + basicos_individual2['MES'].astype(str)).dt.strftime('%Y-%m')

    #     if(basicos_individual2.shape[0] < 10):
    #         continue
    #     else:
    #         df_model = pd.concat([basicos_individual2,test_df], axis=0, ignore_index=True)

    #         meses_basicos = [1,2,3,4,5,6,7,8,9,10,11,12]
    #         basicos_forecast = {}

    #         basicos_results = pd.DataFrame()
    #         basicos1['MES'] = pd.to_datetime(basicos1['MES_AÑO']).dt.month

    #         for mes in meses_basicos:
    #             if mes in basicos1['MES'].unique():
    #                 basicos_results = pd.concat([basicos_results, XGB_model(basicos1[basicos1['MES'] == mes])])
                    
    #             else:
    #                 basicos_forecast[mes] = 0


In [20]:
basicos_individual = basicos[(basicos['Mundo'] == 'NIÑA' ) &
                                        (basicos['Tipo de Artículo'] == 'CAMISA MANGA LARGA')]
basicos_individual2 = basicos_individual.groupby(['MES', 'AÑO'])[
        'UNIDADES'].sum().to_frame().sort_values(by=['AÑO', 'MES']).reset_index()


basicos_individual2['MES_AÑO'] = pd.to_datetime(basicos_individual2['AÑO'].astype(str) + 
                            '-' + basicos_individual2['MES'].astype(str)).dt.strftime('%Y-%m')

In [21]:
df_model = pd.concat([basicos_individual2,test_df], axis=0, ignore_index=True)

meses_basicos = [1,2,3,4,5,6,7,8,9,10,11,12]
basicos_forecast = {}

basicos_individual2['MES'] = pd.to_datetime(basicos_individual2['MES_AÑO']).dt.month

basicos_results = pd.DataFrame()

for mes in meses_basicos:
    if mes in basicos_individual2['MES'].unique():
        basicos_results = pd.concat([basicos_results, XGB_model(df_model[df_model['MES'] == mes])])
    else:
        basicos_forecast[mes] = 0
        basicos_results = pd.concat([basicos_results, pd.DataFrame.from_dict({'MES_AÑO': [(mes)], 'UNIDADES': [0]}).set_index('MES_AÑO')])
        

In [22]:
basicos.Mundo.value_counts()

NIÑO          879023
BEBÉ NIÑO     746489
NIÑA          596578
BEBÉ NIÑA     466217
PRIMI NIÑO    287046
PRIMI NIÑA    218452
OTROS          27062
Name: Mundo, dtype: int64

In [24]:
basicos_results = pd.DataFrame()
basicos_results_final = pd.DataFrame()
c = 0

mundos = ['NIÑA','NIÑO','BEBÉ NIÑO','BEBÉ NIÑA','PRIMI NIÑO','PRIMI NIÑA']
tipos = ['SANDALIA','VESTIDO MANGA LARGA','CAMISA MANGA LARGA','OVERALL CORTO', 'OVERALL CORTO','DENIM']


for mundo in mundos:
# for mundo in basicos['Mundo'].unique():
    output_file = f"basicos_resultados_{mundo}.xlsx"
    basicos_results_final = pd.DataFrame()
    for tipo in basicos['Tipo de Artículo'].unique():

    # for tipo in tipos:

        basicos_individual = basicos[(basicos['Mundo'] == mundo ) &
                                        (basicos['Tipo de Artículo'] == tipo)]


        basicos_individual2 = basicos_individual.groupby(['MES', 'AÑO'])[
        'UNIDADES'].sum().to_frame().sort_values(by=['AÑO', 'MES']).reset_index()


        basicos_individual2['MES_AÑO'] = pd.to_datetime(basicos_individual2['AÑO'].astype(str) + 
                                    '-' + basicos_individual2['MES'].astype(str))#.dt.strftime('%Y-%m')
        
        if(len(basicos_individual2) >= 15):

            df_model = pd.concat([basicos_individual2,test_df], axis=0, ignore_index=True)
    
            meses_basicos = [1,2,3,4,5,6,7,8,9,10,11,12]
            basicos_forecast = {}

            basicos_individual2['MES'] = pd.to_datetime(basicos_individual2['MES_AÑO']).dt.month
            
            # basicos_results = pd.concat([basicos_results, XGB_model(df_model)])

            for mes in meses_basicos:
                if mes in basicos_individual2['MES'].unique():
                    basicos_results = pd.concat([basicos_results, XGB_model(df_model[df_model['MES'] == mes])])
                else:
                    basicos_forecast[mes] = 0
                    basicos_results = pd.concat([basicos_results, pd.DataFrame.from_dict({'MES_AÑO': [mes], 'UNIDADES': [0]}).set_index('MES_AÑO')])

            basicos_results2 = basicos_results.reset_index()
            
            
            basicos_results2['MES_AÑO'] = pd.to_datetime(basicos_results2['MES_AÑO'].astype(str))#.dt.strftime('%Y-%m')
            basicos_results2['mundo'] = mundo
            basicos_results2['tipo'] = tipo
            basicos_results2['cv'] = calcular_Cv(basicos_individual2)
            basicos_results2 = basicos_results2.sort_values(by='MES_AÑO')
                        
            if(len(basicos_results)> 10):
                basicos_results_final =  basicos_results_final.append(basicos_results2, ignore_index= True)
                # print(basicos_results_final.columns,basicos_results_final.index)
                basicos_results = pd.DataFrame()

        else:
            continue

        with pd.ExcelWriter(output_file) as writer:
            basicos_results_final.to_excel(writer, sheet_name=mundo, index=False)
    # data_frame2.to_excel(writer, sheet_name="Vegetables", index=False)
    # data_frame3.to_excel(writer, sheet_name="Baked Items", index=False)

ValueError: Given date string 4 not likely a datetime present at position 3

In [ ]:
basicos_results_final.to_excel('BASICOS_NIÑA.xslx')

In [ ]:
Ñ

In [ ]:
basicos_results

In [ ]:
basicos_individual2['MES_AÑO'] = pd.to_datetime(basicos_individual2['MES_AÑO'])
basicos_individual2[['MES_AÑO','UNIDADES']]

In [ ]:
results = basicos_individual2[['MES_AÑO','UNIDADES']].merge(
    basicos_results.reset_index().rename(columns={'UNIDADES':'FORECAST'}),
    on= 'MES_AÑO'
)

In [ ]:
results['diferencia'] = np.ceil(abs(results['UNIDADES']- results['FORECAST']))

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error as MAPE
print('MAPE ES  ..............',   round(MAPE(results['UNIDADES'], results['FORECAST']),2))

In [ ]:
results

In [ ]:
results.set_index('MES_AÑO').plot()